In [73]:
!rm diabetes.sqlite
!rm diabetes.sqlite-journal

PWD = !pwd
PWD = PWD[0]
PWD

rm: impossibile rimuovere 'diabetes.sqlite-journal': File o directory non esistente


'/home/lore/Documents/GitHub/DataMining/ex1'

In [74]:
#The Code field is deciphered as follows:
CODES = """
33 = Regular insulin dose
34 = NPH insulin dose
35 = UltraLente insulin dose
48 = Unspecified blood glucose measurement
56 = Unkown
57 = Unspecified blood glucose measurement
58 = Pre-breakfast blood glucose measurement
59 = Post-breakfast blood glucose measurement
60 = Pre-lunch blood glucose measurement
61 = Post-lunch blood glucose measurement
62 = Pre-supper blood glucose measurement
63 = Post-supper blood glucose measurement
64 = Pre-snack blood glucose measurement
65 = Hypoglycemic symptoms
66 = Typical meal ingestion
67 = More-than-usual meal ingestion
68 = Less-than-usual meal ingestion
69 = Typical exercise activity
70 = More-than-usual exercise activity
71 = Less-than-usual exercise activity
72 = Unspecified special event
"""

PATIENT_PATH = "dataset/data-{}"
PATIENTS = [1,70]

In [75]:

from pony.orm import *
from datetime import datetime, timedelta
import pandas as pd
from io import StringIO
import numpy as np
import traceback


DB = Database()
DB.bind('sqlite', f"{PWD}/diabetes.sqlite", create_db=True)

In [76]:
def has_subseq(seq, subseq):
    i,j = 0,0
    while i < len(seq) and j < len(subseq):
        if seq[i] == subseq[j]:
            j += 1
        i += 1
    return j == len(subseq)

In [77]:
class Label(DB.Entity):
    code = PrimaryKey(int)
    name = Required(str)
    labels = Set("EventLabel")

    @staticmethod
    def load_codes():
        df_code = pd.read_csv(
            StringIO(CODES), sep="=", header=None, names=["code", "label"]
        )
        for row in df_code.iterrows():
            Label(code=row[1].to_dict()["code"], name=row[1].to_dict()["label"])
        DB.commit()

    def to_dataframe(self):
        return pd.DataFrame(
            [e.event.to_dict() | e.to_dict() | self.to_dict() for e in self.labels]
        )


class EventLabel(DB.Entity):
    event = Required("Event")
    label = Required(Label)
    value = Required(str)
    PrimaryKey(event, label)


class Event(DB.Entity):
    patient_id = Required(int)
    vt = Required(datetime)
    labels = Set(EventLabel)
    PrimaryKey(patient_id, vt)

    @staticmethod
    def event_slices(patient, days=1, stride=1):
        # exercise: a function that produce the window-stride sequences of a given patient
        slices = []
        start_day = min(e.vt for e in Event if e.patient_id == patient)
        end_day = max(e.vt for e in Event if e.patient_id == patient)

        while start_day + timedelta(days=days) <= end_day:
            events = list(
                select(
                    e.labels.label.code
                    for e in Event
                    if e.patient_id == patient
                    and e.vt >= start_day
                    and e.vt < start_day + timedelta(days=days)
                )
            )
            if len(events) > 0:
                slices.append(events)
            start_day = start_day + timedelta(days=stride)

        return slices

    @staticmethod
    def count_seq(seq, patient, days=1, stride_days=1):
        slices = Event.event_slices(patient, days, stride_days)
        print(slices)
        count = 0
        for s in slices:
            if has_subseq(s, seq):
                count += 1
        return count

    @staticmethod
    def load_partecipant(id):
        skipped = 0
        df = pd.read_csv(PATIENT_PATH.format(str(id).zfill(2)), sep="\t", header=None)
        for r in df.iterrows():
            d = r[1].to_dict()
            try:
                vt = datetime.strptime(d[0] + " " + d[1], "%m-%d-%Y %H:%M")
                event = Event.get(patient_id=id, vt=vt)
                if event is None:
                    event = Event(patient_id=id, vt=vt)
                new_label = EventLabel(event=event, label=Label[d[2]], value=str(d[3]))
                event.labels.add(new_label)
            except Exception as e:
                skipped = skipped + 1
                print(e)
                #print(traceback.format_exc())
                print(f"partecipant {id} cannot insert {d}")
        DB.commit()

    @staticmethod
    def load_partecipants():
        i = PATIENTS[0]
        while i <= PATIENTS[1]:
            Event.load_partecipant(i)
            i = i + 1

    def full_dict(self):
        return [
            {
                "vt": self.vt,
                "patient": self.patient_id,
                "label": label.label.name,
                "code": label.label.code,
            }
            for label in self.labels
        ]

    @staticmethod
    def full_dataset():
        r = []
        for e in list(Event.select()):
            r += e.full_dict()
        return pd.DataFrame(r)

In [78]:
DB.generate_mapping(create_tables=True)

In [79]:
Label.load_codes()

In [80]:
Event.load_partecipants()

Cannot create EventLabel: instance with primary key Event[3,datetime.datetime(1990, 7, 30, 5, 28)], Label[33] already exists
partecipant 3 cannot insert {0: '07-30-1990', 1: '05:28', 2: 33, 3: 3}
Cannot create EventLabel: instance with primary key Event[3,datetime.datetime(1990, 8, 1, 16, 28)], Label[65] already exists
partecipant 3 cannot insert {0: '08-01-1990', 1: '16:28', 2: 65, 3: 0}
Cannot create EventLabel: instance with primary key Event[8,datetime.datetime(1990, 8, 14, 10, 8)], Label[65] already exists
partecipant 8 cannot insert {0: '08-14-1990', 1: '10:08', 2: 65, 3: 0}
Cannot create EventLabel: instance with primary key Event[8,datetime.datetime(1990, 8, 15, 7, 26)], Label[33] already exists
partecipant 8 cannot insert {0: '08-15-1990', 1: '07:26', 2: 33, 3: 6}
Cannot create EventLabel: instance with primary key Event[12,datetime.datetime(1990, 8, 30, 6, 29)], Label[33] already exists
partecipant 12 cannot insert {0: '08-30-1990', 1: '06:29', 2: 33, 3: 8}
Cannot create Even

In [81]:
df = Event.full_dataset()
df.head()

,vt,patient,label,code
0,1991-04-21 09:09:00,1,NPH insulin dose,34
1,1991-04-21 09:09:00,1,Regular insulin dose,33
2,1991-04-21 09:09:00,1,Pre-breakfast blood glucose measurement,58
3,1991-04-21 17:08:00,1,Regular insulin dose,33
4,1991-04-21 17:08:00,1,Pre-supper blood glucose measurement,62


In [82]:
df = Event.full_dataset()

print(Event.count_seq([33], 1, 4, 1))

[[33, 34, 58, 62, 48], [33, 62, 34, 58, 48], [33, 62, 34, 58, 48], [33, 62, 48, 34, 58], [33, 62, 48, 34, 58], [33, 62, 34, 58, 48, 65, 60], [33, 34, 58, 62, 48, 65, 60], [33, 62, 48, 34, 58, 65, 60], [65, 33, 60, 62, 48, 34, 58], [33, 60, 62, 65, 48, 34, 58], [65, 33, 60, 34, 58, 62, 48], [33, 65, 62, 34, 58, 60, 48], [33, 60, 62, 48, 34, 58, 65], [33, 48, 65, 34, 58, 62], [33, 34, 58, 65, 48, 62], [33, 62, 34, 58, 48], [33, 34, 58, 62, 48, 65], [33, 62, 48, 34, 58, 65], [33, 34, 58, 65, 62, 48], [33, 65, 62, 48, 34, 58], [33, 62, 48, 34, 58, 65], [33, 34, 58, 62, 48, 65], [33, 62, 48, 34, 58, 65], [33, 65, 62, 48, 34, 58], [33, 62, 34, 58, 48], [33, 62, 48, 34, 58], [33, 62, 34, 58], [33, 62, 34, 58, 60], [33, 34, 58, 62, 60], [33, 62, 34, 58, 60, 48], [33, 60, 34, 58, 62, 48], [33, 62, 34, 58, 60, 48, 65], [60, 33, 62, 48, 34, 58, 65], [33, 62, 48, 34, 58, 65, 60], [65, 33, 62, 48, 34, 58, 60], [33, 34, 58, 62, 48, 60, 65], [33, 60, 48, 34, 58, 62, 65], [33, 60, 34, 58, 62, 65, 48],